# Account - AVR Guest Experience PROD

## Database Summary

In [7]:
Import-Module AWS.Tools.Common
Import-Module AWS.Tools.DynamoDBv2
Import-Module AWS.Tools.RDS

$Profiles   = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match 'DevOps-AVR-Guest-Experience-PROD' }
$Regions    = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"

function Get-AWSDBInformation {
    [CmdletBinding()]
    param (

        $Profiles,
        $Regions,
        $RegexGroup             = 'AccountNumber',
        $RegexGroupAccountName  = 'AccountName',
        $RegexAccountNumber     = "(?<$RegexGroup>\d+):db.+$",
        $RegexAccountNumberDDb  = "(?<$RegexGroup>\d+):table.+$", 
        $RegexAccountName       = "(?<PreFix>DevOps-)(?<$RegexGroupAccountName>.+)"
    )
    
    begin {

        $Summary    = @()
        
    }
    
    process {

        $Summary    = @()
        foreach ($Profile in $Profiles) {

            foreach ($Region in $Regions) {
                
                $RDSInstances   = Get-RDSDBInstance -ProfileName $Profile.ProfileName -Region $Region
                $ddbTables      = Get-DDBTableList -ProfileName $Profile.ProfileName -Region $Region

                switch ([string]::IsNullOrEmpty($RDSInstances)) {
                    $true { 
                        if ($ddbTables) {
                            
                            $Table = $ddbTables | Select-Object -First 1
                            $ddbTable = Get-DDBTable -TableName $Table -ProfileName $Profile.ProfileName -Region $Region
                            $AccountID  = (($ddbTable.TableArn | Select-String -Pattern $RegexAccountNumberDDb).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                        }else {

                            $AccountID = 0
                        }
                    }
                    $false {

                        $RDSInstance   = $RDSInstances | Select-Object -First 1
                        $AccountID  = (($RDSInstance.DBInstanceArn | Select-String -Pattern $RegexAccountNumber).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                    }
                    default {

                        $AccountID = 0
                    }
                }

                $Sum = [PSCustomObject]@{

                    Account             = (($Profile.ProfileName | select-string -Pattern $RegexAccountName).Matches.Groups | Where-Object { $_.Name -eq $RegexGroupAccountName }).Value
                    AccountID           = $AccountID
                    Region              = $Region
                    RDSInstances        = ($RDSInstances.DBInstanceIdentifier | Measure-Object).Count
                    DynamoDBTables      = $ddbTables.Count
                    
                }

                $condition = $Sum.RDSInstances + $Sum.DynamoDBTables
                if ($condition -gt 0) {
                    $Summary += $Sum
                }
            }

        }
        
    }
    
    end {

        $Summary
        
    }
}

$Summary    = Get-AWSDBInformation $Profiles $Regions
$Summary    | Format-Table -AutoSize


Account                   AccountID    Region    RDSInstances DynamoDBTables
-------                   ---------    ------    ------------ --------------
AVR-Guest-Experience-PROD 957578851978 eu-west-2            1              2



### DynamoDB Tables

In [11]:
$ddbTables  = @()
$ddbTableList   = @()
$Regions | ForEach-Object   {

    $ddbTableList   = Get-DDBTableList -ProfileName $Profiles.ProfileName -Region $_
    $ddbBackupList  = Get-DDBBackupList -ProfileName $Profiles.ProfileName -Region $_

    foreach ($ddbTable in $ddbTableList) {
        
        $Table = Get-DDBTable -TableName $ddbTable -ProfileName $Profiles.ProfileName -Region $_

        if ($ddbBackupList) {
            
            $BackupAvailable = $ddbBackupList.TableName.Contains("$($Table.TableName)")
        }
        else {

            $BackupAvailable = $false
        }
        

        $Sum = [PSCustomObject]@{

            Region              = $_
            TableName           = $Table.TableName
            Backup              = $BackupAvailable
            CreationDateTime    = $Table.CreationDateTime
            ItemCount           = $Table.ItemCount
            # TableSizeBytes      = $Table.TableSizeBytes
            SizeMB         = "$([math]::round($($Table.TableSizeBytes /1MB), 0)) MB"
            Status         = $Table.TableStatus
            # TableId             = $Table.TableId
            # KeySchema           = $Table.KeySchema
            # GlobalSecondaryIndexes = $Table.GlobalSecondaryIndexes
            
        }
        $ddbTables += $Sum
    }
    
    

}
$ddbTables | Format-Table -AutoSize


Region    TableName       Backup CreationDateTime      ItemCount SizeMB Status
------    ---------       ------ ----------------      --------- ------ ------
eu-west-2 terraform-lock   False 3/16/2021 2:55:01 PM         49 0 MB   ACTIVE
eu-west-2 terraform-locks  False 10/19/2021 1:14:49 PM        68 0 MB   ACTIVE



### RDS Instances

In [12]:
$RDSInstances = Get-RDSDBInstance -ProfileName $Profiles.ProfileName -Region eu-west-2

$Inst = @()
$Snap = @()
$Snapshots = @()
foreach ($Instance in $RDSInstances) {
    
    $DBInstanceArn      = $Instance.DBInstanceArn
    # $Tags               = Get-RDSTagForResource -ResourceName $DBInstanceArn -ProfileName $Profile.ProfileName
    $Shots          = Get-RDSDBSnapshot -ProfileName $Profiles.ProfileName -DBInstanceIdentifier $DBInstanceArn
    #$DBClusterSnapshots = Get-RDSDBClusterSnapshot -ProfileName $Profile.ProfileName -DBClusterIdentifier $DBInstanceArn

    $Sum = [PSCustomObject]@{

        #DBName                  = $Instance.DBName
        DBInstanceIdentifier    = $Instance.DBInstanceIdentifier
        Engine                  = $Instance.Engine
        EngineVersion           = $Instance.EngineVersion
        InstanceCreateTime      = $Instance.InstanceCreateTime
        #DBInstanceClass         = $Instance.DBInstanceClass
        BackupRetention         = $Instance.BackupRetentionPeriod
        #StorageThroughput       = $Instance.StorageThroughput
        #Tags                    = $Tags
        Snapshots               = ($Shots | Measure-Object).Count
        #DBClusterSnapshots      = ($DBClusterSnapshots | Measure-Object).Count
        
    }
    $Inst += $Sum
    $Snapshots += $Shots
    
}

$Snapshots | ForEach-Object {

    $Sum = [PSCustomObject]@{

        DBInstanceIdentifier    = $_.DBInstanceIdentifier
        DBSnapshotIdentifier    = $_.DBSnapshotIdentifier
        SnapshotCreateTime      = $_.SnapshotCreateTime
        Engine                  = $_.Engine
        SnapshotType            = $_.SnapshotType
        Status                  = $_.Status
        
    }
    $Snap += $Sum
}
$Inst | Format-Table -AutoSize


DBInstanceIdentifier Engine  EngineVersion InstanceCreateTime   BackupRetention Snapshots
-------------------- ------  ------------- ------------------   --------------- ---------
places-api           mariadb 10.6.16       4/11/2023 3:36:31 PM               7         9



#### Snapshots

##### places-api

In [13]:
$DB = 'places-api'
$Snap | Where-Object {$_.DBInstanceIdentifier -eq $DB} | Select-Object DBSnapshotIdentifier, SnapshotCreateTime, SnapshotType, Status |
Sort-Object SnapshotCreateTime -Descending  |Format-Table -AutoSize


DBSnapshotIdentifier            SnapshotCreateTime   SnapshotType Status
--------------------            ------------------   ------------ ------
rds:places-api-2024-07-13-03-12 7/13/2024 4:12:54 AM automated    available
rds:places-api-2024-07-12-03-13 7/12/2024 4:14:01 AM automated    available
rds:places-api-2024-07-11-03-12 7/11/2024 4:12:36 AM automated    available
rds:places-api-2024-07-10-03-12 7/10/2024 4:12:53 AM automated    available
rds:places-api-2024-07-09-03-14 7/9/2024 4:14:26 AM  automated    available
rds:places-api-2024-07-08-03-13 7/8/2024 4:13:36 AM  automated    available
rds:places-api-2024-07-07-03-13 7/7/2024 4:13:19 AM  automated    available
rds:places-api-2024-07-06-03-12 7/6/2024 4:12:57 AM  automated    available
rds:places-api-2024-07-05-03-16 7/5/2024 4:16:41 AM  automated    available

